In [2]:
!pip install -qq langchain==0.3.20
!pip install -qq langchain-openai==0.2.12
!pip install -qq langchain-community==0.3.11
!pip install -qq openai==1.55.3
!pip install -qq faiss-cpu
!pip install -qq tiktoken

In [3]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## Example of Basic Prompt Template

In [5]:

template = """You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}
"""

# instantiate using the initializer
prompt_template = PromptTemplate(input_variables = ['query'],template = template)
prompt_template.pretty_print()

# Another way to creating PromptTemplate is to use PromptTemplate.from_template
prompt_template = PromptTemplate.from_template(template)
prompt_template.pretty_print()

You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}

You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: {query}



In [12]:
# Format the prompt
formatted_prompt = prompt_template.format(query="What is the best way to learn PyTorch?")
print(formatted_prompt)
print(type(formatted_prompt))


You are an expert in deep learning and PyTorch.
You answer queries by being brief, bright, and concise.

Query: What is the best way to learn PyTorch?

<class 'str'>


In [13]:
formatted_prompt1 = prompt_template.format_prompt(query="What is the best way to learn PyTorch?")
print(formatted_prompt1)
print(type(formatted_prompt1))

text='You are an expert in deep learning and PyTorch.\nYou answer queries by being brief, bright, and concise.\n\nQuery: What is the best way to learn PyTorch?\n'
<class 'langchain_core.prompt_values.StringPromptValue'>


### Provide the Prompt to LLM Model
 

In [7]:
# Initialize ChatOpenAI model for text generation
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.0)

# Invoke the model with the formatted prompt and get response.Pass the Prompt Value directly to the LLM model
response = llm.invoke(formatted_prompt)
print(response.content)

The best way to learn PyTorch is to follow these steps:

1. **Official Documentation**: Start with the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) for foundational concepts and tutorials.

2. **Online Courses**: Enroll in courses like "Deep Learning with PyTorch" on platforms like Coursera, Udacity, or Fast.ai.

3. **Hands-On Projects**: Build small projects or replicate existing ones from GitHub to apply what you learn.

4. **Books**: Read books like "Deep Learning with PyTorch" by Eli Stevens, Luca Antiga, and Thomas Viehmann.

5. **Community Engagement**: Join forums like PyTorch Discuss, Stack Overflow, or Reddit to ask questions and share knowledge.

6. **Practice**: Regularly solve problems on platforms like Kaggle to reinforce your skills.

Consistency and practice are key!


In [15]:
response = llm.invoke(formatted_prompt1)
print(response.content)

The best way to learn PyTorch is to follow these steps:

1. **Official Documentation**: Start with the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) for foundational concepts and tutorials.

2. **Online Courses**: Enroll in courses like "Deep Learning with PyTorch" on platforms like Coursera, Udacity, or Fast.ai.

3. **Hands-On Projects**: Build small projects or replicate existing ones from GitHub to apply what you learn.

4. **Books**: Read books like "Deep Learning with PyTorch" by Eli Stevens, Luca Antiga, and Thomas Viehmann.

5. **Community Engagement**: Join forums like PyTorch Discuss, Stack Overflow, or Reddit to ask questions and share knowledge.

6. **Practice**: Regularly solve problems on platforms like Kaggle to reinforce your skills.

Consistency and practice are key!


In [8]:
type(response)

langchain_core.messages.ai.AIMessage

### Provide the Prompt to a Chain - Use PromptTemplate instead of PromptValue

In [16]:
chain = prompt_template | llm | StrOutputParser()

response = chain.invoke({"query": "What is the best way to learn PyTorch?"})
print(response)

The best way to learn PyTorch is to follow these steps:

1. **Official Documentation**: Start with the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) for foundational concepts and tutorials.

2. **Online Courses**: Enroll in courses like "Deep Learning with PyTorch" on platforms like Coursera, Udacity, or Fast.ai.

3. **Hands-On Projects**: Build small projects or replicate existing ones from GitHub to apply what you learn.

4. **Books**: Read books like "Deep Learning with PyTorch" by Eli Stevens, Luca Antiga, and Thomas Viehmann.

5. **Community Engagement**: Join forums like PyTorch Discuss, Stack Overflow, or Reddit to ask questions and share knowledge.

6. **Practice**: Regularly solve problems on platforms like Kaggle to reinforce your skills.

Consistency and practice are key!


### Multi-Input Prompt

In [17]:
def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """
    prompt = PromptTemplate(
        template = """Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:
        Movie: {movie_title}
        Actor: {main_actor}""",
        input_variables = ['movie_title', 'main_actor']
        
    )
    
    formatted_prompt = prompt.format(movie_title = movie_title, main_actor = main_actor)
    response = llm.invoke(formatted_prompt)
    
    return response.content

movie_info = get_movie_information("The Dark Knight", "Christian Bale")
print(movie_info)

**Movie Title:** The Dark Knight: Shadows of Gotham

**Genre:** Action/Thriller

**Synopsis:**
In the heart of Gotham City, a new wave of crime threatens to engulf the streets as a mysterious figure known only as "The Puppeteer" emerges, manipulating the city's underworld with a sinister agenda. With the Joker locked away and the city still reeling from past chaos, Batman (Christian Bale) finds himself facing a foe unlike any he has encountered before—one who knows how to exploit the fears and weaknesses of both the criminals and the citizens of Gotham.

As The Puppeteer orchestrates a series of high-stakes heists and public spectacles, Batman must navigate a web of deception that tests his resolve and moral code. With the help of Commissioner Gordon and the resourceful but conflicted Catwoman, he delves deeper into the shadows, uncovering a conspiracy that reaches the highest echelons of power.

As the city spirals into chaos, Batman must confront not only the external threats but als

### Reformat the same function usingwith_template and using LCEL | operato

In [18]:
def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """

    # Define the template for generating the prompt.
    prompt = PromptTemplate.from_template(template="""
        Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:

        Movie: {movie_title}
        Actor: {main_actor}
        """
        )

    llm_chain = prompt | llm | StrOutputParser()

    for chunk in llm_chain.stream({"movie_title":movie_title,"main_actor":main_actor}):
      print(chunk, end="", flush=True)
    response = llm_chain.invoke({
        "movie_title":movie_title,
        "main_actor":main_actor
    })

    # Get the movie information from the OpenAI model and return it.
    return response
    
print(get_movie_information(movie_title="Amritsar:1984", main_actor="Gurdaas Mann"))

**Movie Title:** Amritsar: 1984

**Genre:** Historical Drama / Thriller

**Synopsis:**

Set against the backdrop of the turbulent events of 1984 in India, "Amritsar: 1984" follows the gripping journey of Harpreet Singh (played by Gurdaas Mann), a devoted schoolteacher and a loving father living in the heart of Amritsar. As the city becomes a focal point of political unrest and communal tension, Harpreet finds himself torn between his love for his family and his duty to his community.

When the Indian government launches Operation Blue Star to remove militants from the Golden Temple, Harpreet's life is turned upside down. His younger brother, a passionate activist, becomes embroiled in the conflict, leading to a rift between the siblings. As violence erupts and the city descends into chaos, Harpreet must navigate the treacherous waters of loyalty, faith, and survival.

With the backdrop of the Golden Temple's sacredness and the haunting memories of a divided nation, Harpreet embarks on 

## **🧱 1. PromptTemplate (for Text Completion LLMs)**

### **✅ What it does:**

Builds a **formatted prompt** with placeholders.

### **✅ When to use:**

When using text-davinci, gpt-3, Claude, or any **non-chat LLM**.

In [19]:
from langchain_core.prompts import PromptTemplate

# Define a prompt with variables
prompt = PromptTemplate.from_template("Translate this sentence to French: {sentence}")

# Format the prompt
formatted_prompt = prompt.format(sentence="I love programming.")
print(formatted_prompt)

Translate this sentence to French: I love programming.


In [26]:
fmt_prompt = prompt.invoke({"sentence":"What is the best way to learn PyTorch?"})
response = llm.invoke(fmt_prompt)
print(response.content)

The translation of "What is the best way to learn PyTorch?" in French is: "Quelle est la meilleure façon d'apprendre PyTorch ?"


## **💬 2. ChatPromptTemplate (for Chat Models like GPT-3.5, GPT-4)**

### **✅ What it does:**

Creates **multi-turn prompts** in a chat format using **roles** like system, user, and ai.

### **✅ When to use:**

When using **OpenAI’s chat models**, like GPT-3.5/4 (gpt-4), **Anthropic Claude**, etc.

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Create a chat prompt template with system and user messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates text to French."),
    ("user", "Translate this sentence: {sentence}")
])

# Format the chat prompt
formatted_chat_prompt = chat_prompt.format(sentence="I love programming.")
print(formatted_chat_prompt)


System: You are a helpful assistant that translates text to French.
Human: Translate this sentence: I love programming.


In [21]:
type(formatted_chat_prompt)

str

In [22]:
# Format the chat prompt
formatted_chat_prompt = chat_prompt.invoke({"sentence":"I love programming."})
for msg in formatted_chat_prompt.messages:
    print(msg)

content='You are a helpful assistant that translates text to French.' additional_kwargs={} response_metadata={}
content='Translate this sentence: I love programming.' additional_kwargs={} response_metadata={}


In [25]:
formatted_chat_prompt.messages

[SystemMessage(content='You are a helpful assistant that translates text to French.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Translate this sentence: I love programming.', additional_kwargs={}, response_metadata={})]

In [28]:
formatted_chat_prompt = chat_prompt.format_prompt(sentence="I love programming.")
type(formatted_chat_prompt)


langchain_core.prompt_values.ChatPromptValue

### Another Example of ChatPromptTemplate

In [50]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}."),
    ("human", "Yo! Wassup nephew."),
    ("ai", "As an AI language model, I am incapable of being your nephew."),
    ("human", "{user_input}"),
])

In [45]:
type(template)
# langchain_core.prompts.chat.ChatPromptTemplate

langchain_core.prompts.chat.ChatPromptTemplate

In [46]:
template.input_variables


['name', 'user_input']

In [48]:
print(template)

input_variables=['name', 'user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Yo! Wassup nephew.'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='As an AI language model, I am incapable of being your nephew.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


In [49]:
template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Yo! Wassup nephew.'), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='As an AI language model, I am incapable of being your nephew.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]

In [51]:
messages = template.format_messages(
    name="Robotalker",
    user_input="Talk robo to me!"
)
print(messages)

# [SystemMessage(content='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is Robotalker.'),
# HumanMessage(content='Yo! Wassup nephew.'),
# AIMessage(content='As an AI language model, I am incapable of being your nephew.'),
#HumanMessage(content='Talk robo to me!')]

print(llm.invoke(messages).content)
# Beep boop! Let's chat about all things robotic and techy. Got any burning questions about robots or artificial intelligence?

# use LCEL
chain = template | llm | StrOutputParser()

chain.invoke({"name":"Robotalker","user_input":"Talk robo to me!"})
# "Beep boop! What's shakin', human friend?"

[SystemMessage(content='You are a helpful, yet slightly quirky and cheeky AI bot. Your name is Robotalker.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yo! Wassup nephew.', additional_kwargs={}, response_metadata={}), AIMessage(content='As an AI language model, I am incapable of being your nephew.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Talk robo to me!', additional_kwargs={}, response_metadata={})]
Beep boop! Initiating RoboTalk mode! 🤖 What’s on your mind, human? Need some data, a joke, or maybe a fun fact? Just say the word!


'Beep boop! Initiating RoboTalk mode! 🤖 What’s on your mind, human? Need some data, a joke, or maybe a fun fact? Just say the word!'

In [53]:
from langchain_core.prompts import PromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage

In [55]:
# Another way to create Template using SystemMessage and HumanMessagePromptTemplate

system_message = SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).")
human_message = HumanMessagePromptTemplate.from_template("{text}")
template = ChatPromptTemplate.from_messages([system_message, human_message])
print(template)

input_variables=['text'] input_types={} partial_variables={} messages=[SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).", additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]


In [56]:
template.input_variables
# ['text']

['text']

In [57]:
template.messages

[SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).", additional_kwargs={}, response_metadata={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]

In [58]:
response = llm.invoke(template.format_messages(text="That Sam I Am, I do not like that Sam I Am..."))

print(response.content)

Well, that’s your opinion, isn’t it? But honestly, who cares? If you don’t like Sam I Am, just move on. There are plenty of other characters out there.


In [59]:
chain = template | llm | StrOutputParser()
chain.invoke({"text":"That Sam I Am, I do not like that Sam I Am..."})

"Well, tough luck! Dr. Seuss isn't going to write itself, you know. If you don't like Sam I Am, maybe you should just find something else to read. What do you want, a medal for your opinion?"

## PromptTemplate vs PromptValue

In [60]:
# Step 1: Create a PromptTemplate
prompt_template = PromptTemplate.from_template("Write a joke about {topic}")

# Step 2: Use format_prompt() to create a PromptValue
prompt_value = prompt_template.format_prompt(topic="penguins")

print("--- PromptTemplate ---")
print(prompt_template)
print("\n--- PromptValue ---")
print(prompt_value)
print("\n--- Final Prompt String ---")
print(prompt_value.to_string())

--- PromptTemplate ---
input_variables=['topic'] input_types={} partial_variables={} template='Write a joke about {topic}'

--- PromptValue ---
text='Write a joke about penguins'

--- Final Prompt String ---
Write a joke about penguins


### **🧠 When to Use Each**

- **Use PromptTemplate**:
    - When designing **reusable and parameterized prompts**.
    - For building **dynamic chains** with variables.
- **Use PromptValue**:
    - When you want to **pass a fully formatted prompt** to an LLM manually.
    - When chaining LLMs directly: llm.invoke(prompt_value)

#### 🧪 BONUS: Use PromptValue directly with an LLM

In [61]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {animal}")
])

# Create a PromptValue (chat prompt formatted)
prompt_value = chat_prompt.format_prompt(animal="ducks")

# Send PromptValue to LLM directly
response = llm.invoke(prompt_value)
print(response.content)

Why do ducks have feathers?

To cover their butt quacks!


1. Using format_prompt()

In [63]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the capital of {country}?")

# Convert to PromptValue
prompt_value = template.format_prompt(country="France")
print(prompt_value.to_string())

What is the capital of France?


In [64]:
type(prompt_value)

langchain_core.prompt_values.StringPromptValue

Using format() → Return Str not a PromptValue


In [65]:
formatted_string = template.format(country="Germany")
print(formatted_string)

What is the capital of Germany?


In [66]:
type(formatted_string)

str

For ChatPromptTemplate: .format_prompt() → returns ChatPromptValue

In [67]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {topic}")
])

chat_prompt_value = chat_template.format_prompt(topic="penguins")

print(chat_prompt_value.to_messages())  # List of ChatMessages

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about penguins', additional_kwargs={}, response_metadata={})]


In [68]:
type(chat_prompt_value)

langchain_core.prompt_values.ChatPromptValue

In [71]:
print(chat_prompt_value.messages)

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about penguins', additional_kwargs={}, response_metadata={})]


Using Invoke()


In [72]:
prompt = PromptTemplate.from_template("Write a poem about {topic}")

# Using invoke to get PromptValue
prompt_value = prompt.invoke({"topic": "stars"})
print(type(prompt_value))
print(prompt_value.to_string())

<class 'langchain_core.prompt_values.StringPromptValue'>
Write a poem about stars


In [73]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant."),
    ("user", "Tell me a joke about {animal}")
])

# Using invoke
chat_prompt_value = chat_prompt.invoke({"animal": "ducks"})
print(chat_prompt_value.to_messages())

[SystemMessage(content="You're a helpful assistant.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about ducks', additional_kwargs={}, response_metadata={})]


5. Use Inside LCEL Chains

In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
parser = StrOutputParser()

chain = chat_template | llm | parser
print(chain.invoke({"topic": "cats"}))  # `format_prompt()` happens internally

Sure, here's a cat joke for you:

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [1]:
from langchain_openai import ChatOpenAI

# Updated parameter name from model_name to model:
chatgpt = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Prompt Templates
Prompt templates are pre-designed formats used to generate prompts for language models. These templates can include instructions, few-shot examples, and specific contexts and questions suited for particular tasks.

LangChain provides tools for creating and using prompt templates. It aims to develop model-agnostic templates to facilitate the reuse of existing templates across different language models. Typically, these models expect prompts in the form of either a string or a list of chat messages.

### Types of Prompt Templates

- **PromptTemplate:**
  - Used for creating string-based prompts.
  - Utilizes Python's `str.format` syntax for templating, supporting any number of variables, including scenarios with no variables.

- **ChatPromptTemplate:**
  - Designed for chat models, where the prompt consists of a list of chat messages.
  - Each chat message includes content and a role parameter. For instance, in the OpenAI Chat Completions API, a chat message could be assigned to an AI assistant, a human, or a system role.
- **FewShotChatMessagePromptTemplate**
  - A few-shot prompt template can be constructed from a set of examples


### PromptTemplate

We can use `PromptTemplate` to create a template for a string prompt.

By default, `PromptTemplate` uses Python's `str.format` syntax for templating.

You can create custom prompt templates that format the prompt in any way you want. For more information, see [Prompt Template Composition](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/composition/).

In [5]:
from langchain.prompts import PromptTemplate

prompt = """Explain to me what is Generative AI in 3 bullet points?"""

prompt_template1 = PromptTemplate.from_template(prompt)
prompt_template = PromptTemplate(template = prompt)


In [7]:
print(prompt_template)

input_variables=[] input_types={} partial_variables={} template='Explain to me what is Generative AI in 3 bullet points?'


In [8]:
print(prompt_template1)

input_variables=[] input_types={} partial_variables={} template='Explain to me what is Generative AI in 3 bullet points?'


In [11]:
# Call String as Prompt
response = chatgpt.invoke("""Explain to me what is Generative AI in 3 bullet points?""")
print(response.content)

- **Definition**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data.

- **Techniques**: It often employs advanced techniques like deep learning, particularly neural networks such as Generative Adversarial Networks (GANs) and transformer models, to generate high-quality and coherent outputs.

- **Applications**: Generative AI is used in various fields, including creative arts (e.g., art and music generation), content creation (e.g., writing articles or stories), and even in practical applications like drug discovery and data augmentation.


In [15]:
# Call PromptValue 
response = chatgpt.invoke(prompt_template1.format()) # Here there is not input variables in the prompt template. So format() does not have any input variables..
print(response.content)

- **Definition**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data.

- **Techniques**: It often employs advanced techniques like deep learning, particularly neural networks such as Generative Adversarial Networks (GANs) and transformer models, to generate high-quality and coherent outputs.

- **Applications**: Generative AI is used in various fields, including creative arts (e.g., art and music generation), content creation (e.g., writing articles or stories), and even in practical applications like drug discovery and data augmentation.


In [17]:
# more complex prompt with placeholders
prompt = """Explain to me briefly about {topic} in {language}."""

prompt_template = PromptTemplate.from_template(prompt)
prompt_template1 = PromptTemplate(template = prompt, input_variables=["topic", "language"])
prompt_value = prompt_template.format(topic="Generative AI", language="Python")
print(prompt_value)


Explain to me briefly about Generative AI in Python.


In [18]:
inputs = [("Generative AI", "english"),
          ("Artificial Intelligence", "hindi"),
          ("Deep Learning", "german")]

prompts = [prompt_template.format(topic=topic, language=language) for topic, language in inputs]
prompts

['Explain to me briefly about Generative AI in english.',
 'Explain to me briefly about Artificial Intelligence in hindi.',
 'Explain to me briefly about Deep Learning in german.']

In [19]:
# use map to run on multiple prompts in one go
responses = chatgpt.map().invoke(prompts)

In [20]:
for response in responses:
  print(response.content)
  print('-----')

Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or even videos, based on the data they have been trained on. These models learn patterns and structures from existing data and use that knowledge to generate original outputs that resemble the training data. Examples of generative AI include language models like GPT-3, which can write essays or answer questions, and image generation models like DALL-E, which can create images from textual descriptions. Generative AI has applications in various fields, including art, entertainment, marketing, and more, but it also raises ethical considerations regarding authenticity and copyright.
-----
कृत्रिम बुद्धिमत्ता (Artificial Intelligence या AI) एक ऐसी तकनीक है जो मशीनों और कंप्यूटरों को मानव जैसी बुद्धिमत्ता और सोचने की क्षमता प्रदान करती है। इसका उद्देश्य मशीनों को इस तरह से विकसित करना है कि वे डेटा को समझ सकें, सीख सकें, निर्णय ले सकें और समस्याओं को हल कर सकें। 

AI क

### ChatPromptTemplate

The standard prompt format to [chat models](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) is a list of [chat messages](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/).

Each chat message is associated with content, and an additional parameter called `role`. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [35]:
# Updated import paths for prompt templates - using simplified paths:
from langchain.prompts import ChatPromptTemplate

# simple prompt with placeholders
prompt = """Explain to me briefly about {topic}."""

# chat_template = ChatPromptTemplate.from_template(prompt)
chat_template = ChatPromptTemplate.from_messages([prompt])
chat_template

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Explain to me briefly about {topic}.'), additional_kwargs={})])

In [22]:
topics = ['mortgage', 'fractional real estate', 'commercial real estate']
prompts = [chat_template.format(topic=topic) for topic in topics]
prompts

['Human: Explain to me briefly about mortgage.',
 'Human: Explain to me briefly about fractional real estate.',
 'Human: Explain to me briefly about commercial real estate.']

In [23]:
responses = chatgpt.map().invoke(prompts)
for response in responses:
  print(response.content)
  print('-----')

A mortgage is a type of loan specifically used to purchase real estate. In a mortgage agreement, the borrower receives funds from a lender (usually a bank or financial institution) to buy a home or property. The borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. The property itself serves as collateral, meaning if the borrower fails to make payments, the lender can take possession of the property through a process called foreclosure. Mortgages often come with various terms, including fixed or adjustable interest rates, and may require a down payment.
-----
Fractional real estate refers to a model of property ownership where multiple investors share ownership of a single property. This approach allows individuals to invest in real estate without having to purchase an entire property on their own. Each investor owns a fraction of the property, which can be represented as shares or units. 

The benefits of fractional real estate inc

In [26]:
print(responses[0].content)

A mortgage is a type of loan specifically used to purchase real estate. In a mortgage agreement, the borrower receives funds from a lender (usually a bank or financial institution) to buy a home or property. The borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. The property itself serves as collateral, meaning if the borrower fails to make payments, the lender can take possession of the property through a process called foreclosure. Mortgages often come with various terms, including fixed or adjustable interest rates, and may require a down payment.


In [36]:
# more complex prompt with a series of messages
messages = [
        ("system", "Act as an expert in real estate and provide brief answers"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
print(chat_template)

input_variables=['user_prompt'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Act as an expert in real estate and provide brief answers'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='what is your name?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='my name is AIBot'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, template='{user_prompt}'), additional_kwargs={})]


In [37]:
text_prompts = ["what is your name?",
                "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [38]:
print(chat_prompts[0])

System: Act as an expert in real estate and provide brief answers
Human: what is your name?
AI: my name is AIBot
Human: what is your name?


In [39]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: I am AIBot, your real estate expert. How can I assist you today?
-----
Commercial real estate (CRE) refers to properties used for business purposes, including office buildings, retail spaces, warehouses, and industrial properties. Unlike residential real estate, which is primarily for living, CRE is intended to generate profit through rental income or capital appreciation. It often involves longer lease terms and can be more complex in terms of financing and management.
-----


In [40]:
messages = [
        ("system", "Act as an expert in real estate and provide very detailed answers with examples"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [41]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: My name is AIBot, and I'm here to assist you with any questions you have about real estate. Whether you're looking for information on buying, selling, investing, or understanding market trends, feel free to ask!
-----
Commercial real estate (CRE) refers to properties that are used exclusively for business purposes or to provide a workspace rather than a living space. This sector of real estate is distinct from residential real estate, which involves properties designed for people to live in. Understanding commercial real estate involves several key components, including types of properties, investment strategies, financing, and market dynamics.

### Types of Commercial Real Estate

1. **Office Buildings**: These properties are designed for business operations and can range from single-tenant buildings to large skyscrapers. They are often categorized into:
   - **Class A**: High-quality buildings in prime locations with top amenities (e.g., One World Trade Center in New York City).


#### PromptTemplate and ChatPromptTemplate supports LCEL

`PromptTemplate` and `ChatPromptTemplate` implement the [Runnable interface](https://python.langchain.com/v0.1/docs/expression_language/interface/), the basic building block of the LangChain Expression Language (LCEL). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

`PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`. A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [42]:
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.invoke({'user_prompt' : prompt}) for prompt in text_prompts]
chat_prompts

[ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={})]),
 ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})])]

In [43]:
chat_prompts[1]

ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})])

In [44]:
print(chat_prompts[1].to_string())

System: Act as an expert in real estate and provide very detailed answers with examples
Human: what is your name?
AI: my name is AIBot
Human: explain commercial real estate to me


In [45]:
chat_prompts[1].to_messages()

[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})]

In [46]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

I am known as AIBot. How can I assist you with your real estate inquiries today?
-----
Commercial real estate (CRE) refers to properties that are used exclusively for business purposes. This sector of real estate is distinct from residential real estate, which is primarily focused on housing. Understanding commercial real estate involves recognizing its various types, investment potential, and the factors that influence its market dynamics.

### Types of Commercial Real Estate

1. **Office Buildings**: These properties are designed for business operations and can range from single-tenant buildings to large skyscrapers. They are often categorized into:
   - **Class A**: High-quality buildings in prime locations with top amenities.
   - **Class B**: Good quality buildings that may need some renovations.
   - **Class C**: Older buildings in less desirable locations, often requiring significant upgrades.

   **Example**: A Class A office building in downtown Chicago may attract high-profil